In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

In [ ]:
# Load the ESRGAN model from TensorFlow Hub
model_url = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(model_url)

In [ ]:
import os

# Define the paths for the folders
working_directory = '/kaggle/working'
normal_folder_path = os.path.join(working_directory, 'normal')
pneumonia_folder_path = os.path.join(working_directory, 'pneumonia')

# Create the folders if they don't exist
os.makedirs(normal_folder_path, exist_ok=True)
os.makedirs(pneumonia_folder_path, exist_ok=True)

print("Folders 'normal' and 'pneumonia' created in Kaggle/working directory.")

In [ ]:
def preprocess_image(image_path, target_size=(256, 256)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, 0)
    return image

In [ ]:
input_folder_path = "/kaggle/input/chestxraypneumonia/chest_xray/train/PNEUMONIA"

# Create a folder to store the processed images
output_folder_path = "/kaggle/working/pneumonia"
os.makedirs(output_folder_path, exist_ok=True)

In [ ]:
image_files = [os.path.join(input_folder_path, file) for file in os.listdir(input_folder_path) if file.endswith((".jpg", ".jpeg", ".png"))]


In [ ]:
for image_file in image_files:
    image = preprocess_image(image_file)
    processed_image = model(image)
    processed_image_uint8 = tf.cast(tf.clip_by_value(processed_image[0], 0, 255), tf.uint8)
    
    # Extract the file name without extension
    file_name = os.path.splitext(os.path.basename(image_file))[0]
    output_image_path = os.path.join(output_folder_path, f"{file_name}_processed.jpeg")
    
    # Save the processed image
    tf.io.write_file(output_image_path, tf.image.encode_png(processed_image_uint8))
    
print("Processed Pneumonia images added in folder")


In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)


In [ ]:
zip_dir()